https://www.tensorflow.org/guide/autodiff

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

### Computing gradients

To differentiate automatically, TensorFlow needs to remember what operations happen in what order during the forward pass. Then, during the backward pass, TensorFlow traverses this list of operations in reverse order to compute gradients.

In [2]:
x = tf.Variable(4.0)

with tf.GradientTape() as tape:
    y = x**2

In [3]:
y

<tf.Tensor: shape=(), dtype=float32, numpy=16.0>

In [4]:
dy_dx = tape.gradient(y, x)

dy_dx

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [7]:
w = tf.Variable(tf.random.normal((4, 2)))

w

<tf.Variable 'Variable:0' shape=(4, 2) dtype=float32, numpy=
array([[-0.7118777 , -1.0898987 ],
       [-0.28268456,  0.76275915],
       [-0.40623742, -0.6167519 ],
       [-0.6392075 ,  0.3569129 ]], dtype=float32)>

In [8]:
b = tf.Variable(tf.ones(2, dtype=tf.float32))

b

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>

In [10]:
x = tf.Variable([[10., 20., 30., 40.]], dtype=tf.float32)

x

<tf.Variable 'Variable:0' shape=(1, 4) dtype=float32, numpy=array([[10., 20., 30., 40.]], dtype=float32)>

In [17]:
with tf.GradientTape(persistent=True) as tape:
    y = tf.matmul(x, w) + b
    
    loss = tf.reduce_mean(y**2)

In [18]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

The gradient with respect to each source has the shape of the source

In [19]:
dl_dw

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[ -495.2789  ,    11.301546],
       [ -990.5578  ,    22.603092],
       [-1485.8367  ,    33.90464 ],
       [-1981.1156  ,    45.206184]], dtype=float32)>

In [20]:
dl_db

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-49.52789  ,   1.1301546], dtype=float32)>

In [21]:
layer = tf.keras.layers.Dense(2, activation='relu')

x = tf.constant([[10., 20., 30.]])

In [22]:
with tf.GradientTape() as tape:
    y = layer(x)
    
    loss = tf.reduce_sum(y**2)

grad = tape.gradient(loss, layer.trainable_variables)

In [23]:
grad

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[   0.     ,  429.81946],
        [   0.     ,  859.6389 ],
        [   0.     , 1289.4584 ]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 0.      , 42.981945], dtype=float32)>]

In [24]:
for var, g in zip(layer.trainable_variables, grad):
    print(f'{var.name}, shape: {g.shape}')

dense/kernel:0, shape: (3, 2)
dense/bias:0, shape: (2,)


### Gradients are calculated only with respect to trainable variables

Trainable variable, the value associated with this will be updated during the training process

In [30]:
x1 = tf.Variable(5.0)

x1

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

Trainable has been explicitly set to false

In [31]:
x2 = tf.Variable(5.0, trainable=False)

x2

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

Tensor, not a variable. Gradients are not calculated on Tensors

In [32]:
x3 = tf.add(x1, x2)

x3

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [33]:
x4 = tf.constant(5.0)

x4

<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

In [34]:
with tf.GradientTape() as tape:
    y = (x1**2) + (x2**2) + (x3**2) + (x4**2)

grad = tape.gradient(y, [x1, x2, x3, x4])

grad

[<tf.Tensor: shape=(), dtype=float32, numpy=10.0>, None, None, None]

### Watch constants to calculate gradients with respect to them

tf.GradientTape provides hooks that give the user control over what is or is not watched. To record gradients with respect to a tf.Tensor, you need to call GradientTape.watch(x)

In [56]:
x1 = tf.constant(5.0)

x2 = tf.Variable(3.0)

In [57]:
with tf.GradientTape() as tape:
    tape.watch(x1)
    
    y = (x1**2) + (x2**2)

In [58]:
[dy_dx1, dy_dx2] = tape.gradient(y, [x1, x2])

dy_dx1, dy_dx2

(<tf.Tensor: shape=(), dtype=float32, numpy=10.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=6.0>)

In [59]:
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x1)
    
    y = (x1**2) + (x2**2)

In [60]:
[dy_dx1, dy_dx2] = tape.gradient(y, [x1, x2])

dy_dx1, dy_dx2

(<tf.Tensor: shape=(), dtype=float32, numpy=10.0>, None)

### Gradient tape records operations as they occur

Conditionals are naturally handled. The gradient only connects to the variable that was used.

In [90]:
x = tf.constant(1.0)
x1 = tf.Variable(5.0)
x2 = tf.Variable(3.0)

In [92]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)

    if x > 0.0:
        result = x1**2
    else:
        result = x2**2 

dx1, dx2 = tape.gradient(result, [x1, x2])

dx1, dx2

(<tf.Tensor: shape=(), dtype=float32, numpy=10.0>, None)

In [93]:
x = tf.constant(-1.0)
x1 = tf.Variable(5.0)
x2 = tf.Variable(3.0)

In [94]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)

    if x > 0.0:
        result = x1**2
    else:
        result = x2**2 

dx1, dx2 = tape.gradient(result, [x1, x2])

dx1, dx2

(None, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>)

In [98]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
    z = y * y

    dy_dx = tape.gradient(z, x)
    
print(dy_dx)

None
